In [1]:
import math
import numpy as np
#from MyConvolution import convolve
def myHybridImages(lowImage: np.ndarray, lowSigma: float, highImage: np.ndarray, highSigma: float) -> np.ndarray:
    high_kernel = makeGaussianKernel(highSigma)
    low_kernel = makeGaussianKernel(lowSigma)
    
    #i_1*G_1
    high_pass_image = convolve(highImage,high_kernel)
    low_pass_image = convolve(lowImage,low_kernel)
    
    weightlow = 1
    weighthigh = 1
    
    high_pass_image = highImage - high_pass_image
    
    hybrid_image = low_pass_image*weightlow + high_pass_image*weighthigh
    


    return hybrid_image
 # Your code here.


In [2]:
def makeGaussianKernel(sigma: float) -> np.ndarray:
    kernel_size = int(8*sigma+1)
    if kernel_size % 2 != 0:
        kernel_size += 1
    #center os half of window size
    center = (kernel_size -1)/2
    
    #norma;ise by the total sum
    kernel_sum = 0
    
    kernel = np.zeros((kernel_size,kernel_size))
    coef = 1/(2*np.pi*sigma*sigma)
    for i in range(kernel_size):
        for j in range(kernel_size):
            x = pow((i-center),2)
            y = pow((j-center),2)
            fenzi = -(x+y)
            fenmu = 2*sigma*sigma
            kernel[i][j] = np.exp(fenzi/fenmu)/coef
            kernel_sum += kernel[i][j]
    
    kernel = kernel/kernel_sum
    return kernel

In [3]:
def calculate_convolution(image,kernel):
    kernel = np.rot90(kernel,2)
    kh = int(kernel.shape[0])
    kw = int(kernel.shape[1])
    print(image.shape)
    print(kh)
    print(kw)
    if ((kw % 2 != 0) & (kh % 2 != 0)):
        image_rows = image.shape[0] - kh + 1
        image_cols = image.shape[1] - kw + 1
        new_conv = np.zeros((image_rows,image_cols))
    else:
        image_rows = image.shape[0] - kh + 1
        image_cols = image.shape[1] - kw + 1
        new_conv = np.zeros((image_rows,image_cols))
        for i in range(int(image_rows)):
            for j in range(int(image_cols)):
                result = (image[i : i+kh,j:j+kw]*kernel).sum()
                new_conv[i][j] = result
    return new_conv

In [4]:
def get_padding(padding_2):
    if padding_2 == 0:
        padding_before = 0
        padding_after = 0
        return int(padding_before),int(padding_after)
    elif padding_2 < 2:
        padding_before = 0
        padding_after = 1
        return int(padding_before),int(padding_after)
    elif padding_2 % 2 == 0:
        padding_before = padding_2/2
        padding_after = padding_2/2
        return int(padding_before),int(padding_after)
    else:
        padding_2 -= 1
        padding_before = padding_2/2+1
        padding_after = padding_2/2
        return int(padding_before),int(padding_after)

In [5]:
import numpy as np
from PIL import Image
def convolve(image: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    """
    Convolve an image with a kernel assuming zero-padding of the image to handle the borders
    :param image: the image (either greyscale shape=(rows,cols) or colour shape=(rows,cols,channels))
    :type numpy.ndarray
    :param kernel: the kernel (shape=(kheight,kwidth); both dimensions odd)
    :type numpy.ndarray
    :returns the convolved image (of the same shape as the input image)
    :rtype numpy.ndarray
    """
    # Your code here. You'll need to vectorise your implementation to ensure it runs
    # at a reasonable speed.
    
    stride = 1
    if len(image.shape) == 3:
        rows = image.shape[0]
        cols = image.shape[1]
        channels = image.shape[2]
    else:
        rows = image.shape[0]
        cols = image.shape[1]
        
    kh = kernel.shape[0]
    kw = kernel.shape[1]
        
    rows_2padding = (rows-1)*stride+kh - rows
    cols_2padding = (cols-1)*stride+kw - cols
        
    rows_before,rows_after = get_padding(rows_2padding)
    print("rows_before",rows_before)
    print("rows_after",rows_after)
    cols_before,cols_after = get_padding(cols_2padding)
    print("cols_before",cols_before)
    print("cols_after",cols_after)
    if len(image.shape) == 3:
        image_behind = np.pad(image,((rows_before,rows_after),(cols_before,cols_after),(0,0)), 'constant', constant_values=0)
        image_behind_r = image_behind[:,:,0]
        image_behind_g = image_behind[:,:,1]
        image_behind_b = image_behind[:,:,2]
        result_r = calculate_convolution(image_behind_r,kernel)
        result_g = calculate_convolution(image_behind_g,kernel)
        result_b = calculate_convolution(image_behind_b,kernel)
        convolved_image = np.dstack([result_r,result_g,result_b])
    else:
        image_behind = np.pad(image,((rows_before,rows_after),(cols_before,cols_after)), 'constant', constant_values=0)
        convolved_image = calculate_convolution(image_behind,kernel)
        
    return convolved_image

In [6]:
image = Image.open('./bicycle.bmp')
image = np.array(image)
image2 = Image.open('./motorcycle.bmp')
image2 = np.array(image2)
final = myHybridImages(image,1,image2,2)
Image.fromarray(final.astype(np.uint8))

FileNotFoundError: [Errno 2] No such file or directory: './bicycle.bmp'